# 0.   Packages and Functions

In [ ]:
!pip install pandas_datareader

In [ ]:

#general
import pandas as pd
import numpy as np
import pickle 
from decimal import Decimal


#for dates and times
import random
import time
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar

#prices
import pandas_datareader as web


In [ ]:
#getting nearest date before investment.
def nearest(items, pivot):
    return min([i for i in items if i < pivot], key=lambda x: abs(x - pivot))

In [ ]:


def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [ ]:
def weekend2weekday (transactions, col_date, col_day):


  dates2_BUY=list()
  days2_BUY=list()

  for index,row in transactions.iterrows():
    buy_date = datetime.datetime.strptime(row[col_date], "%Y-%m-%d")
    if row[col_day]=='Sat':
      buy_date2 = buy_date - datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Fri'
      days2_BUY.append(buy_day2)
    elif row[col_day]=='Sun':
      buy_date = datetime.datetime.strptime(row[col_day], "%Y-%m-%d")
      buy_date2 = buy_date + datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Mon'
      days2_BUY.append(buy_day2)
    else:
      buy_date2=row[col_date]
      dates2_BUY.append(buy_date2)
      buy_day2=row[col_day]
      days2_BUY.append(buy_day2)

    transactions['date_BUY_fix']=dates2_BUY
    transactions['day_BUY_fix']=days2_BUY


  return transactions

# 1. Setting investment parameters

## Temporal horizons
 In number of days
 - 1 day to 2 years

In [ ]:
horizons=[1,2,3,4,5,6,7,10,15,20,30,40,50,60,70,80,90,100,120,150,180,210,240,270,300,330,360,420,480,540,600,660,720]


## Amounts in dollars
 - 50 to 50,000 dollars

In [ ]:
amounts=[50,100,200,300,400,500,1000,1500,2000,3000,4000,5000,10000,15000,20000,25000,50000]

# 2. Data extraction

## Companies' stock prices

In [ ]:
#Selecting a list of companies we are interested in analyzing:
companies = ['AAPL',
            'AMZN',
            'MSFT',
            'GOOG',
            'FB',
            'NVDA',
             #automobiles
            'F',
            'GM',
            'HMC',
            'TSLA',
            'TYO',
             'HOG',
             #retail
             'WMT',
             'M',
             'TGT',
             'KSS',
             'COST',
             'BBY',
             #FastMovingConsumerGoods(FMCG)
             'NESN',
             'KO',
             'PEP',
             'ULVR',
             'PG',
             'OR',
            #bank
            'AXP',
            'VISA',
            'SC',
            'DB',
            'UBS',
            'BAC',
            'JPM']

In [ ]:
tech_c=['AAPL','AMZN','MSFT','GOOG','FB','VISA','NVDA']
auto_c=['F','GM','HMC','TSLA','TYO','HOG']
retail_c=[ 'WMT', 'M','TGT','KSS','COST','BBY']
fmcg_c=['NESN','KO','PEP','ULVR','PG', 'OR']
bank_c=['AXP','VISA','SC', 'DB', 'UBS','BAC','JPM']

In [ ]:
#ESG ratios
ESG_ratios=[16.5, 27.4, 14.7, 22.8, 31.6, 12.9, 29.7, 30.6, 28.5, 31.1, 30.3, 16.3, 27.3, 14.5, 14.8, 12.9, 22.3, 12.0, 28.3, 25.1, 17.6, 23.4, 25.1, 20.5, 19.8, 17.4, 26.9, 30.0, 24.8, 26.3, 27.9]

We are going to extract info from 10 years from the companies selected

In [ ]:
#Collecting the data of each company in a list
l_companies_prices=list()
error_companies=list()
error_indexes=list()

for company in companies:
    try:
        prices= web.DataReader(company, data_source='yahoo', start='2010-10-10', end='2020-10-10')
        l_companies_prices.append(prices)
        print(company)
    
    except :
        print('There were  errors when extracting data of  ', company)
        #elimino de la lista esa empresa que no se consiguieron datos.
        error_companies.append(company)
        
        error_company_index=companies.index(company)
        error_indexes.append(error_company_index)


for c in error_companies:
  companies.remove(c)

ESG_ratios_fix=list()

for index in range(0, len(ESG_ratios)):
    if index not in error_indexes:
        ESG_ratios_fix.append(ESG_ratios[index])

AAPL
AMZN
MSFT
GOOG
FB
NVDA
F
GM
HMC
TSLA
TYO
HOG
WMT
M
TGT
KSS
COST
BBY
There were  errors when extracting data of   NESN
KO
PEP
There were  errors when extracting data of   ULVR
PG
OR
AXP
There were  errors when extracting data of   VISA
SC
DB
UBS
BAC
JPM


In [ ]:
companies

['AAPL',
 'AMZN',
 'MSFT',
 'GOOG',
 'FB',
 'NVDA',
 'F',
 'GM',
 'HMC',
 'TSLA',
 'TYO',
 'HOG',
 'WMT',
 'M',
 'TGT',
 'KSS',
 'COST',
 'BBY',
 'KO',
 'PEP',
 'PG',
 'OR',
 'AXP',
 'SC',
 'DB',
 'UBS',
 'BAC',
 'JPM']

In [ ]:
len(ESG_ratios_fix)

28

In [ ]:
len(companies)

28

In [ ]:
print(companies[0])
l_companies_prices[0].columns

AAPL


Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [ ]:
company_pos=companies.index("AAPL")
l_companies_prices[company_pos].loc['2010-10-11','Close']

10.54857063293457

# 3. Random transactions

In [ ]:
transactions=pd.DataFrame()
#we do 40,000 initial transactions

#id
transactions['id']=range(1,40001)

## Companies

In [ ]:
#random company
transactions["company"]=np.random.choice(companies, size=len(transactions))

## Sector

In [ ]:
#sector of the company

random_companies=transactions['company'].tolist()
random_sectors=list()
for comp in random_companies:
    if comp in tech_c:
        sector='TECH'
    elif comp in auto_c:
        sector='AUTO'
    elif comp in retail_c:
        sector='RETAIL'
    elif comp in fmcg_c:
        sector='FMCG'
    elif comp in bank_c:
        sector='BANK'
    
    random_sectors.append(sector)

transactions['sector']=random_sectors

## Horizons

In [ ]:
#random horizon
transactions["horizon"] = np.random.choice(horizons, size=len(transactions))


## Investment Amounts

In [ ]:
#random investment amount
transactions["amount"] = np.random.choice(amounts, size=len(transactions))


## Dates - BUY

In [ ]:
#random buy date
start='2013-10-10'
end='2018-10-10'

dates_buy=list()

for i in range(0, transactions.shape[0]):
    r_date=random_date(start,end,random.random())
    dates_buy.append(r_date)


print(len(dates_buy))
transactions['date_BUY']=dates_buy

40000


### Fixing dates (BUY)

In [ ]:
day_names=list()

for d in dates_buy:

  dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=dt.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_BUY']=day_names


In [ ]:
dates2_BUY=list()
days2_BUY=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
  if row['day_BUY']=='Sat':
    buy_date2 = buy_date - datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Fri'
    days2_BUY.append(buy_day2)
  elif row['day_BUY']=='Sun':
    buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
    buy_date2 = buy_date + datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Mon'
    days2_BUY.append(buy_day2)
  else:
    buy_date2=buy_date
    dates2_BUY.append(buy_date2)
    buy_day2=row['day_BUY']
    days2_BUY.append(buy_day2)


transactions['date_BUY_fix']=dates2_BUY
transactions['day_BUY_fix']=days2_BUY

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [ ]:
#checking for holidays (NYSE is closed)

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_BUY:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_BUY']=l_holidays

In [ ]:
transactions_h=transactions[transactions.holiday_BUY=='Holiday']

transactions=transactions[transactions.holiday_BUY!='Holiday']

## Prices - BUY

In [ ]:
#price when bought

l_buy_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[buy_date,'Adj Close']
  except :
    price=0

  l_buy_prices.append(price)

transactions['price_BUY']=l_buy_prices


## Dates - SELL

In [ ]:
#random sell date
dates_SELL=list()

for index,row in transactions.iterrows():

  buy_date = row['date_BUY_fix']
  sell_date = buy_date + datetime.timedelta(row['horizon'])
  
  dates_SELL.append(sell_date)

transactions['date_SELL']=dates_SELL

### Fixing dates (SELL)

In [ ]:
day_names=list()



for d in dates_SELL:

  # dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=d.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_SELL']=day_names


In [ ]:
dates2_SELL=list()
days2_SELL=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  SELL_date = row['date_SELL']
  if row['day_SELL']=='Sat':
    SELL_date2 = SELL_date - datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Fri'
    days2_SELL.append(SELL_day2)
  elif row['day_SELL']=='Sun':
    SELL_date = row['date_SELL']
    SELL_date2 = SELL_date + datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Mon'
    days2_SELL.append(SELL_day2)
  else:
    SELL_date2=SELL_date
    dates2_SELL.append(SELL_date2)
    SELL_day2=row['day_SELL']
    days2_SELL.append(SELL_day2)


transactions['date_SELL_fix']=dates2_SELL
transactions['day_SELL_fix']=days2_SELL

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [ ]:
transactions

,id,company,sector,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,date_SELL,day_SELL,date_SELL_fix,day_SELL_fix
0,1,KSS,RETAIL,30,300,2016-07-11,Mon,2016-07-11,Mon,,32.497158,2016-08-10,Wed,2016-08-10,Wed
1,2,TYO,AUTO,300,20000,2017-01-05,Thu,2017-01-05,Thu,,14.744580,2017-11-01,Wed,2017-11-01,Wed
2,3,PEP,FMCG,660,500,2018-03-31,Sat,2018-03-30,Fri,,0.000000,2020-01-19,Sun,2020-01-20,Mon
3,4,GM,AUTO,3,10000,2016-07-24,Sun,2016-07-25,Mon,,27.413452,2016-07-28,Thu,2016-07-28,Thu
4,5,TGT,RETAIL,4,100,2014-04-19,Sat,2014-04-18,Fri,,0.000000,2014-04-22,Tue,2014-04-22,Tue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39994,39995,AXP,BANK,180,1500,2016-05-07,Sat,2016-05-06,Fri,,59.775597,2016-11-02,Wed,2016-11-02,Wed
39995,39996,UBS,BANK,360,200,2018-03-31,Sat,2018-03-30,Fri,,0.000000,2019-03-25,Mon,2019-03-25,Mon
39996,39997,NVDA,TECH,4,300,2016-03-11,Fri,2016-03-11,Fri,,31.640415,2016-03-15,Tue,2016-03-15,Tue
39997,39998,FB,TECH,5,500,2016-05-10,Tue,2016-05-10,Tue,,120.500000,2016-05-15,Sun,2016-05-16,Mon


In [ ]:
#checking for holidays (NYSE, NASDAQ is closed)
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_SELL:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_SELL']=l_holidays

## Prices - SELL

In [ ]:
#price when sold

l_sell_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[sell_date,'Adj Close']
  except :
    price=0

  l_sell_prices.append(price)

transactions['price_SELL']=l_sell_prices


In [ ]:
transactions_h=transactions[transactions.holiday_SELL=='Holiday']

transactions=transactions[transactions.holiday_SELL!='Holiday']

In [ ]:
transactions

,id,company,sector,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,date_SELL,day_SELL,date_SELL_fix,day_SELL_fix,holiday_SELL,price_SELL
0,1,KSS,RETAIL,30,300,2016-07-11,Mon,2016-07-11,Mon,,32.497158,2016-08-10,Wed,2016-08-10,Wed,,31.311853
1,2,TYO,AUTO,300,20000,2017-01-05,Thu,2017-01-05,Thu,,14.744580,2017-11-01,Wed,2017-11-01,Wed,,13.657829
3,4,GM,AUTO,3,10000,2016-07-24,Sun,2016-07-25,Mon,,27.413452,2016-07-28,Thu,2016-07-28,Thu,,26.498533
4,5,TGT,RETAIL,4,100,2014-04-19,Sat,2014-04-18,Fri,,0.000000,2014-04-22,Tue,2014-04-22,Tue,,48.878441
5,6,MSFT,TECH,2,500,2015-06-26,Fri,2015-06-26,Fri,,40.755909,2015-06-28,Sun,2015-06-29,Mon,,39.954479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39994,39995,AXP,BANK,180,1500,2016-05-07,Sat,2016-05-06,Fri,,59.775597,2016-11-02,Wed,2016-11-02,Wed,,61.258610
39995,39996,UBS,BANK,360,200,2018-03-31,Sat,2018-03-30,Fri,,0.000000,2019-03-25,Mon,2019-03-25,Mon,,10.561457
39996,39997,NVDA,TECH,4,300,2016-03-11,Fri,2016-03-11,Fri,,31.640415,2016-03-15,Tue,2016-03-15,Tue,,31.561857
39997,39998,FB,TECH,5,500,2016-05-10,Tue,2016-05-10,Tue,,120.500000,2016-05-15,Sun,2016-05-16,Mon,,118.669998


In [ ]:
transactions_ready=transactions[['company','sector','horizon','amount','date_BUY_fix','date_SELL_fix','price_BUY','price_SELL']]

In [ ]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL
0,KSS,RETAIL,30,300,2016-07-11,2016-08-10,32.497158,31.311853
1,TYO,AUTO,300,20000,2017-01-05,2017-11-01,14.744580,13.657829
3,GM,AUTO,3,10000,2016-07-25,2016-07-28,27.413452,26.498533
4,TGT,RETAIL,4,100,2014-04-18,2014-04-22,0.000000,48.878441
5,MSFT,TECH,2,500,2015-06-26,2015-06-29,40.755909,39.954479
...,...,...,...,...,...,...,...,...
39994,AXP,BANK,180,1500,2016-05-06,2016-11-02,59.775597,61.258610
39995,UBS,BANK,360,200,2018-03-30,2019-03-25,0.000000,10.561457
39996,NVDA,TECH,4,300,2016-03-11,2016-03-15,31.640415,31.561857
39997,FB,TECH,5,500,2016-05-10,2016-05-16,120.500000,118.669998


## Filtering transactions with error (Price = 0)

In [ ]:
price_error=transactions_ready[(transactions_ready.price_BUY==0) | (transactions_ready.price_SELL==0)]
print('with errors: ', price_error.shape)
print(transactions_ready.shape)

transactions_ready=pd.concat([transactions_ready, price_error, price_error]).drop_duplicates(keep=False)
print(transactions_ready.shape)

with errors:  (1463, 8)
(36226, 8)
(34713, 8)


# Inflation
Buscamos que supere la tasa de inflacion

In [ ]:
# inflation=pd.read_csv('ratios_inflation.csv', index_col=False)

# inflation['Year'] = pd.to_datetime(inflation['Year'])

# inflation=inflation.set_index('Year')

# inflation

,Tasa_inflacion,IPC
Year,,
1970-01-01 00:00:00.000002011,3.20,2.87
1970-01-01 00:00:00.000002012,2.45,2.93
1970-01-01 00:00:00.000002013,1.41,0.23
1970-01-01 00:00:00.000002014,-0.15,-0.372
1970-01-01 00:00:00.000002015,-0.50,-0.274
1970-01-01 00:00:00.000002016,-0.20,0.675
1970-01-01 00:00:00.000002017,1.96,1.666
1970-01-01 00:00:00.000002018,1.68,1.686
1970-01-01 00:00:00.000002019,0.70,0.408


In [ ]:
# l_inflation=list()
# for index,row in transactions.iterrows():
#     buy_date=row['date_BUY_fix']
#     year=buy_date.year
#     try:
#         inflation=inflation.loc[year,'Tasa_inflacion']
#     except :
#         inflation=0

#     l_inflation.append(inflation)

# transactions['inflation']=l_inflation


<ipython-input-42-fb910abc4a06>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions['inflation']=l_inflation


In [ ]:
# inflation.loc['2019','Tasa_inflacion']

AttributeError: 'int' object has no attribute 'loc'

In [ ]:
# transactions.inflation.value_counts()

# Real Profit/Loss (Rentabilidad Real)

In [ ]:
transactions_ready['nominal_return']=(transactions_ready['price_SELL']-transactions_ready['price_BUY'])/transactions_ready['price_BUY']

In [ ]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return
0,KSS,RETAIL,30,300,2016-07-11,2016-08-10,32.497158,31.311853,-0.036474
1,TYO,AUTO,300,20000,2017-01-05,2017-11-01,14.744580,13.657829,-0.073705
3,GM,AUTO,3,10000,2016-07-25,2016-07-28,27.413452,26.498533,-0.033375
5,MSFT,TECH,2,500,2015-06-26,2015-06-29,40.755909,39.954479,-0.019664
7,BBY,RETAIL,3,20000,2016-08-29,2016-09-01,34.658939,34.232590,-0.012301
...,...,...,...,...,...,...,...,...,...
39991,FB,TECH,240,100,2016-07-27,2017-03-24,123.339996,140.339996,0.137830
39993,KSS,RETAIL,80,1500,2018-07-25,2018-10-12,66.574463,65.971695,-0.009054
39994,AXP,BANK,180,1500,2016-05-06,2016-11-02,59.775597,61.258610,0.024810
39996,NVDA,TECH,4,300,2016-03-11,2016-03-15,31.640415,31.561857,-0.002483


In [ ]:
transactions_ready['investment']=''
investments=list()
for index, row in transactions_ready.iterrows():
  if row['nominal_return']>0:
    inv='GOOD'
    investments.append(inv)
  elif row['nominal_return']<0:
    inv='BAD'
    investments.append(inv)
  else:
    inv='0'
    investments.append(inv)

transactions_ready['investment']=investments

In [ ]:
transactions_ready.shape

(34713, 10)

In [ ]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment
0,KSS,RETAIL,30,300,2016-07-11,2016-08-10,32.497158,31.311853,-0.036474,BAD
1,TYO,AUTO,300,20000,2017-01-05,2017-11-01,14.744580,13.657829,-0.073705,BAD
3,GM,AUTO,3,10000,2016-07-25,2016-07-28,27.413452,26.498533,-0.033375,BAD
5,MSFT,TECH,2,500,2015-06-26,2015-06-29,40.755909,39.954479,-0.019664,BAD
7,BBY,RETAIL,3,20000,2016-08-29,2016-09-01,34.658939,34.232590,-0.012301,BAD
...,...,...,...,...,...,...,...,...,...,...
39991,FB,TECH,240,100,2016-07-27,2017-03-24,123.339996,140.339996,0.137830,GOOD
39993,KSS,RETAIL,80,1500,2018-07-25,2018-10-12,66.574463,65.971695,-0.009054,BAD
39994,AXP,BANK,180,1500,2016-05-06,2016-11-02,59.775597,61.258610,0.024810,GOOD
39996,NVDA,TECH,4,300,2016-03-11,2016-03-15,31.640415,31.561857,-0.002483,BAD


In [ ]:
transactions_ready.investment.value_counts()

GOOD    20182
BAD     14165
0         366
Name: investment, dtype: int64

### Expected return

In [ ]:
l_returns=list()
l_stdDev=list()

for company in companies:
  company_pos=companies.index(company)
  prices_r=l_companies_prices[company_pos]['Adj Close']

  # Calculting log returns and std deviation
  percent_change = prices_r.pct_change()
  returns = np.log(1+percent_change)

  meanReturn_daily = returns.mean()
  stdDev_daily = returns.std()    

  meanReturn_year= meanReturn_daily * 365
  stdDev_year= stdDev_daily *stdDev_daily* 365

  l_returns.append(meanReturn_year)
  l_stdDev.append(stdDev_year)
  


In [ ]:
companies_returns=list(zip(companies,l_returns))
companies_stddev=list(zip(companies,l_stdDev))

In [ ]:
companies_returns

[('AAPL', 0.37006814429896073),
 ('AMZN', 0.4447562102652766),
 ('MSFT', 0.34874318892447237),
 ('GOOG', 0.2509879900028765),
 ('FB', 0.33424316922218367),
 ('NVDA', 0.582100040715823),
 ('F', -0.036749878404302924),
 ('GM', 0.028801730711424594),
 ('HMC', -0.05782888896173675),
 ('TSLA', 0.6779141428282532),
 ('TYO', -0.2306144640410261),
 ('HOG', 0.016042203169368604),
 ('WMT', 0.174738668786327),
 ('M', -0.1474254373640375),
 ('TGT', 0.20006492374571294),
 ('KSS', -0.08109459573535757),
 ('COST', 0.28972100435924225),
 ('BBY', 0.19485826232397122),
 ('KO', 0.12239288649463223),
 ('PEP', 0.1512073670209265),
 ('PG', 0.16589283819663966),
 ('OR', -0.00748706568595733),
 ('AXP', 0.1710617622201037),
 ('SC', -0.024354755713299828),
 ('DB', -0.23159903755060685),
 ('UBS', -0.056055549945980554),
 ('BAC', 0.11337527794846122),
 ('JPM', 0.17508913733547116)]

In [ ]:
#prueba de modificacion desde jupyter notebook.

# Ratios

## ESG Ratios

In [ ]:
len(ESG_ratios_fix)

28

In [ ]:
companies[3]

'GOOG'

In [ ]:
ESG_ratios_fix

[16.5,
 27.4,
 14.7,
 22.8,
 31.6,
 12.9,
 29.7,
 30.6,
 28.5,
 31.1,
 30.3,
 16.3,
 27.3,
 14.5,
 14.8,
 12.9,
 22.3,
 12.0,
 25.1,
 17.6,
 25.1,
 20.5,
 19.8,
 26.9,
 30.0,
 24.8,
 26.3,
 27.9]

In [ ]:
transactions_ready['ESG_ranking']=0

for index, row in transactions_ready.iterrows():
    company=row['company']
    ind_comp=companies.index(company)
    ESG_ratio= ESG_ratios_fix[ind_comp]
    transactions_ready.loc[index, 'ESG_ranking']=ESG_ratio
            

In [ ]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment,ESG_ranking
0,KSS,RETAIL,30,300,2016-07-11,2016-08-10,32.497158,31.311853,-0.036474,BAD,12.9
1,TYO,AUTO,300,20000,2017-01-05,2017-11-01,14.744580,13.657829,-0.073705,BAD,30.3
3,GM,AUTO,3,10000,2016-07-25,2016-07-28,27.413452,26.498533,-0.033375,BAD,30.6
5,MSFT,TECH,2,500,2015-06-26,2015-06-29,40.755909,39.954479,-0.019664,BAD,14.7
7,BBY,RETAIL,3,20000,2016-08-29,2016-09-01,34.658939,34.232590,-0.012301,BAD,12.0
...,...,...,...,...,...,...,...,...,...,...,...
39991,FB,TECH,240,100,2016-07-27,2017-03-24,123.339996,140.339996,0.137830,GOOD,31.6
39993,KSS,RETAIL,80,1500,2018-07-25,2018-10-12,66.574463,65.971695,-0.009054,BAD,12.9
39994,AXP,BANK,180,1500,2016-05-06,2016-11-02,59.775597,61.258610,0.024810,GOOD,19.8
39996,NVDA,TECH,4,300,2016-03-11,2016-03-15,31.640415,31.561857,-0.002483,BAD,12.9


## PE Ratio 

In [ ]:
pe_ratios_collection= pickle.load(open('pe_ratios_collection.p', "rb" ))

In [ ]:
pe_ratios_collection['AAPL']

,Date,Stock_Price,TTM_Net_EPS,PE_Ratio
0,NaT,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN
2,2021-01-01,127.14,,39.00
3,2020-09-01,115.61,$3.26,35.46
4,2020-06-01,90.88,$3.29,27.64
5,2020-03-01,63.18,$3.19,19.82
6,2019-12-01,72.78,$3.17,23.00
7,2019-09-01,55.35,$2.96,18.68
8,2019-06-01,48.72,$2.93,16.62
9,2019-03-01,46.58,$2.97,15.67


In [ ]:
l_PE_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=pe_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            PE_ratio_str=pe_ratios_collection[company][pe_ratios_collection[company]['Date']==date].PE_Ratio.values[0]
            PE_ratio=float(PE_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PE_ratio= Decimal('nan')
            
    except:
        PE_ratio=Decimal('nan')
    
    l_PE_ratios.append(PE_ratio)

In [ ]:
transactions_ready['PE_ratio']=l_PE_ratios

In [ ]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment,ESG_ranking,PE_ratio
0,KSS,RETAIL,30,300,2016-07-11,2016-08-10,32.497158,31.311853,-0.036474,BAD,12.9,11.1
1,TYO,AUTO,300,20000,2017-01-05,2017-11-01,14.744580,13.657829,-0.073705,BAD,30.3,NaN
3,GM,AUTO,3,10000,2016-07-25,2016-07-28,27.413452,26.498533,-0.033375,BAD,30.6,3.1
5,MSFT,TECH,2,500,2015-06-26,2015-06-29,40.755909,39.954479,-0.019664,BAD,14.7,27.33
7,BBY,RETAIL,3,20000,2016-08-29,2016-09-01,34.658939,34.232590,-0.012301,BAD,12.0,9.71
...,...,...,...,...,...,...,...,...,...,...,...,...
39991,FB,TECH,240,100,2016-07-27,2017-03-24,123.339996,140.339996,0.137830,GOOD,31.6,54.94
39993,KSS,RETAIL,80,1500,2018-07-25,2018-10-12,66.574463,65.971695,-0.009054,BAD,12.9,11.59
39994,AXP,BANK,180,1500,2016-05-06,2016-11-02,59.775597,61.258610,0.024810,GOOD,19.8,11.33
39996,NVDA,TECH,4,300,2016-03-11,2016-03-15,31.640415,31.561857,-0.002483,BAD,12.9,26.56


## EPS Ratio 

In [ ]:
import re
trim = re.compile(r'[^\d.,]+')
mystring = 'USD 10.99'
result = trim.sub('', mystring)
print(result)
result=float(result)
print(type(result))
# '10.99'

10.99
<class 'float'>


In [ ]:
l_EPS_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']

    try:
        ratios_company=pe_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),row['date_BUY_fix'])
    
        try:
            EPS_ratio_str=pe_ratios_collection[company][pe_ratios_collection[company]['Date']==date].TTM_Net_EPS.values[0]
            EPS_ratio_str=trim.sub('',EPS_ratio_str)
            print(EPS_ratio_str)
            EPS_ratio=float(EPS_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            EPS_ratio= Decimal('nan')
            
    except:
        EPS_ratio=Decimal('nan')
    
    l_EPS_ratios.append(EPS_ratio)

3.07
7.81
1.46
3.07
0.40
5.39
3.57
5.03
0.27
1.58
1.44
5.55
1.76
3.82
2.14
1.10
2.73
0.12
7.81
0.42
1.44
5.49
2.14
0.92
1.65
1.05
1.85
0.62
3.28
2.83
0.54
4.41
3.77
1.85
1.95
5.16
5.86
1.90
0.77
1.59
23.09
1.67
3.28
1.28
2.16
1.56
3.26
1.93
3.24
4.37
0.77
1.26
3.77
5.49
2.08
1.56
0.78
2.96
1.12
5.50
1.52
2.05
1.75
17.96
1.46
0.42
3.57
2.99
4.41
0.63
4.88
1.44
2.98
0.30
3.18
5.19
0.74
5.41
4.29
1.34
5.22
23.74
0.07
3.29
3.79
4.97
1.90
2.43
3.08
0.40
1.86
0.07
1.09
1.26
0.29
1.75
4.36
3.37
0.27
3.22
1.58
0.94
3.76
1.01
3.76
4.96
4.26
1.87
4.60
0.91
3.07
4.03
1.38
6.49
2.14
4.89
0.75
1.91
1.58
4.07
1.06
0.83
2.70
1.86
3.27
2.11
1.74
4.42
5.59
0.58
1.26
2.41
0.93
1.93
0.91
0.33
1.52
2.96
1.76
4.23
3.07
3.74
0.78
5.80
2.66
2.35
1.17
1.62
2.61
6.94
4.17
1.06
2.92
7.81
3.76
2.14
4.78
2.94
0.42
5.29
4.43
4.36
0.62
1.03
2.20
4.29
3.66
1.24
4.07
0.77
1.93
2.73
2.59
1.85
2.72
5.19
3.82
5.70
3.50
2.05
5.32
3.71
0.38
3.79
4.07
1.12
5.04
5.35
5.72
3.76
0.96
1.56
3.63
5.12
5.34
3.83
25.81
1.12
3.67
3

3.28
6.94
7.10
1.19
4.01
1.26
6.46
3.20
22.75
0.89
5.63
5.32
0.99
1.71
4.17
3.95
0.98
3.71
2.94
2.64
5.03
3.79
3.49
4.27
5.49
0.70
2.90
2.05
6.85
5.05
25.81
5.19
5.29
1.26
2.17
0.52
3.76
0.92
5.34
1.09
0.75
2.20
5.54
3.62
0.85
1.68
3.18
4.39
1.12
2.73
4.52
5.46
2.14
4.23
1.52
1.86
1.50
2.25
3.32
5.46
3.10
2.86
0.95
4.88
0.07
1.59
1.06
0.38
4.84
1.12
1.06
2.70
4.88
1.56
2.25
4.78
1.90
3.57
3.37
0.50
1.26
2.14
4.17
0.27
1.93
3.96
1.30
5.20
0.27
5.59
26.58
1.55
0.58
2.96
2.70
1.08
1.07
2.69
6.63
0.48
0.32
0.17
1.34
2.66
1.62
6.46
2.79
23.56
1.08
1.44
1.06
6.04
1.93
1.49
5.70
5.66
1.75
3.07
4.48
5.06
2.63
3.87
1.76
23.09
2.58
2.41
4.01
1.61
2.96
0.83
1.51
2.96
5.50
5.72
3.07
1.37
8.07
4.72
3.87
3.53
3.67
1.43
2.64
1.03
2.79
2.20
4.65
1.99
3.28
3.30
2.52
5.19
1.71
3.75
5.06
1.06
1.75
7.81
5.26
4.37
4.17
4.88
1.71
0.63
0.92
1.75
3.44
26.58
3.22
2.73
5.49
4.26
1.58
3.20
2.73
2.14
2.13
2.13
3.82
3.79
3.08
0.43
0.38
1.26
3.28
23.56
2.58
3.21
2.61
3.17
1.49
1.07
3.15
2.94
1.91
0.17
2.79
0.95
0.0

4.29
6.67
4.32
0.70
0.77
3.13
2.44
3.40
4.01
4.90
0.50
4.03
5.46
0.32
5.74
3.30
0.85
3.28
1.05
3.99
2.30
2.59
4.64
3.74
2.63
2.65
0.74
0.54
2.05
0.42
5.04
1.39
3.86
4.37
2.05
2.26
4.45
3.76
4.56
0.43
0.96
3.44
2.09
1.87
3.66
5.66
1.26
6.08
3.53
2.12
2.65
4.07
2.30
1.74
2.54
3.62
0.93
5.72
1.83
0.94
1.75
3.69
5.90
2.36
0.92
4.67
5.26
2.69
0.12
2.58
1.03
1.95
1.93
1.55
2.93
5.04
5.86
2.96
4.27
5.63
3.82
3.74
0.89
1.37
6.49
27.83
1.31
3.50
4.29
2.32
1.68
1.99
1.37
3.10
3.93
0.77
0.59
4.26
5.19
2.83
27.55
0.43
0.96
0.77
2.32
4.02
1.55
4.88
1.31
2.38
3.87
0.77
0.50
1.58
2.99
0.98
1.55
3.84
5.06
1.52
2.96
3.39
3.50
2.96
3.51
3.42
2.54
3.44
3.39
2.58
5.77
2.41
4.37
4.26
3.07
1.30
1.08
5.77
1.71
1.97
2.27
3.90
2.44
2.14
5.99
2.20
3.96
1.12
0.93
0.42
0.27
3.82
0.63
0.95
1.55
1.55
3.76
3.24
4.07
0.93
1.69
5.16
4.97
6.71
2.73
1.24
4.29
1.10
5.74
1.17
0.96
4.27
2.89
2.59
4.52
2.91
23.56
1.02
5.20
5.90
4.36
4.39
1.55
23.09
1.19
4.52
1.66
1.50
0.19
5.12
1.17
1.01
25.81
4.39
1.17
4.88
0.96
1.01
5.03


2.08
3.82
0.85
3.07
1.68
1.75
1.55
6.46
7.49
5.34
3.90
1.80
2.96
0.54
0.27
1.80
5.69
6.71
0.21
4.32
1.50
3.71
3.96
5.70
5.50
3.62
1.58
2.08
4.01
2.99
2.11
3.07
2.55
3.69
2.92
6.04
2.48
27.33
1.87
2.15
5.50
0.07
4.42
3.32
0.63
21.10
4.72
3.27
5.59
4.87
2.36
3.57
5.50
5.19
2.07
6.46
1.61
1.12
4.23
3.82
1.03
4.82
1.09
4.27
5.29
0.63
0.89
2.36
3.08
25.81
1.54
0.85
1.39
1.06
3.90
4.10
3.50
1.06
3.74
1.61
3.21
1.62
4.45
1.67
2.58
2.58
3.50
3.94
2.36
4.57
5.50
0.97
2.12
5.22
0.42
1.69
5.29
4.60
2.15
2.39
2.55
1.93
1.71
2.15
2.25
0.88
3.40
5.40
1.68
4.91
25.81
2.36
27.83
2.30
1.10
3.22
3.40
2.36
0.38
0.28
3.13
5.19
5.19
2.96
3.74
3.13
0.83
1.61
0.89
0.27
1.69
2.55
17.96
5.50
4.78
5.63
1.17
0.91
1.58
0.29
4.01
2.96
4.88
0.21
23.74
1.12
1.12
0.92
2.99
4.85
5.63
5.59
1.03
3.75
4.57
2.72
3.53
1.05
3.66
5.05
2.49
1.24
3.75
5.28
2.20
4.87
3.27
3.76
20.97
3.79
1.03
2.67
1.10
5.46
6.19
1.74
5.46
0.29
0.96
0.85
5.42
4.29
7.94
27.55
3.51
2.30
2.03
1.56
2.26
3.20
3.66
3.10
1.90
6.71
0.47
4.91
1.86
23.56


0.89
3.22
5.12
3.18
5.19
4.79
1.39
3.95
2.08
1.12
3.77
3.43
4.17
0.77
2.96
6.30
2.98
1.08
4.88
2.86
21.10
4.26
5.59
0.96
5.71
3.79
27.55
3.32
5.35
2.61
4.72
0.22
3.07
1.51
2.70
1.58
21.10
1.14
0.54
1.06
0.58
3.18
2.55
0.92
3.69
3.55
0.75
2.55
1.06
0.85
7.02
2.27
1.52
0.21
0.95
5.46
4.17
2.27
5.63
0.92
5.99
1.30
1.46
6.82
1.12
5.37
2.14
1.52
3.18
2.27
1.06
5.20
2.83
0.98
1.50
1.46
6.76
3.20
4.29
1.75
0.17
5.14
0.07
2.98
5.72
0.27
3.15
1.58
2.86
2.20
1.49
1.06
2.64
2.16
1.38
3.73
2.16
1.80
23.56
5.40
3.43
3.86
0.19
2.05
0.85
6.46
5.35
0.30
1.55
23.09
1.66
0.37
3.77
4.87
6.85
5.00
4.63
2.15
3.17
4.33
0.27
2.70
0.91
1.93
1.31
3.90
3.77
1.75
2.70
3.57
1.51
1.55
12.61
0.59
2.16
3.44
4.03
2.39
1.95
1.03
2.52
5.35
1.05
0.99
5.63
0.70
0.88
27.55
5.77
1.61
0.43
2.63
5.42
0.89
5.63
6.00
5.05
29.54
3.21
4.79
8.07
5.29
6.85
23.74
4.39
1.34
0.95
4.90
0.85
2.65
2.05
6.01
1.49
1.03
0.62
2.54
0.26
4.66
1.05
2.96
3.30
5.16
3.18
6.76
0.89
1.62
5.50
1.24
5.46
1.68
5.35
1.03
5.80
2.43
4.39
2.09
5.05
1.55
5

...

2.14
2.61
3.39
7.49
1.80
2.16
3.96
3.07
3.90
0.37
1.12
1.44
3.21
3.94
3.75
1.02
2.25
5.91
3.94
2.49
3.71
0.89
6.04
2.02
0.52
1.81
7.10
3.73
27.55
3.73
1.08
1.26
6.46
2.96
21.10
0.96
4.03
2.41
3.27
21.10
0.07
2.91
3.28
2.07
3.15
3.66
0.94
5.46
1.46
4.89
6.01
2.05
2.35
5.37
4.67
3.50
3.53
5.49
4.29
0.78
2.16
2.55
2.27
4.64
3.69
5.54
0.42
0.88
0.74
2.20
1.54
3.73
0.89
3.20
2.70
3.21
1.90
0.27
5.19
4.10
4.23
2.55
2.70
2.02
0.95
5.39
1.52
5.70
5.49
0.89
5.03
2.99
4.87
2.32
3.50
5.19
3.30
3.93
0.62
3.95
5.55
3.90
0.75
0.63
4.02
2.49
0.37
0.75
2.98
3.20
4.27
0.37
1.95
6.85
2.44
4.26
0.38
3.75
2.30
5.22
6.29
3.53
4.16
2.91
5.16
0.19
3.28
1.59
1.10
0.22
2.41
3.87
17.96
5.72
2.08
0.17
3.87
3.74
5.50
0.95
1.64
1.68
4.39
12.61
2.70
2.11
3.69
0.93
7.02
3.74
0.30
1.46
5.86
0.21
5.45
2.99
5.35
1.93
2.36
1.80
3.93
0.91
5.59
3.18
1.03
2.52
2.41
1.65
5.70
1.69
5.53
1.99
3.27
2.58
2.92
0.85
4.32
4.90
0.96
6.82
7.49
6.71
3.82
1.22
0.54
5.32
3.95
3.51
1.58
0.17
1.10
0.91
5.05
3.08
6.94
5.46
1.61
3.28
3.66


5.19
5.16
0.62
6.19
3.28
5.46
2.05
5.69
3.93
27.33
3.71
2.98
2.70
4.91
2.26
4.48
2.05
2.27
1.83
0.37
3.18
0.85
1.28
0.85
3.66
3.13
2.02
1.49
1.05
5.28
3.75
4.27
2.54
0.40
5.46
5.63
0.97
3.44
1.76
1.64
1.12
5.16
1.26
3.99
27.55
0.96
4.57
5.54
5.34
1.34
3.18
2.44
1.52
1.03
3.82
1.07
2.89
2.20
1.28
1.64
5.37
4.41
3.82
7.81
0.22
7.94
4.78
0.62
0.97
1.05
4.72
3.87
1.65
1.22
4.01
0.38
0.83
5.69
3.90
29.87
2.11
3.82
5.29
3.93
6.49
6.82
3.82
5.14
5.50
2.67
5.80
4.88
1.67
2.03
1.80
1.62
0.70
4.33
4.29
0.64
3.71
0.52
3.17
8.73
1.01
0.69
3.99
3.37
2.55
5.69
1.49
0.77
0.42
1.34
2.96
0.27
2.30
3.17
0.21
3.49
4.37
3.75
0.92
5.34
3.51
0.38
0.29
7.49
1.75
4.17
1.83
6.85
4.33
1.43
5.40
3.49
5.40
5.14
1.26
3.87
1.22
1.67
1.03
4.85
4.10
1.06
3.32
1.28
4.39
3.37
3.30
1.83
1.99
4.65
3.49
1.69
2.71
2.25
5.69
1.83
5.52
5.22
1.86
3.32
3.10
0.77
7.10
0.75
2.64
5.03
2.43
4.37
1.44
5.90
1.06
2.96
1.12
2.14
1.93
5.12
4.88
1.49
1.75
5.86
1.80
0.85
1.55
0.85
1.74
1.26
0.88
1.09
2.90
6.76
2.20
6.82
3.79
3.18
2.20
1.

0.12
2.15
4.39
1.26
2.07
3.90
1.37
2.67
5.99
4.88
2.32
1.64
1.38
0.54
5.55
2.02
4.48
2.14
5.30
1.17
3.53
1.52
6.94
2.25
2.54
1.49
2.89
5.32
1.02
7.49
6.29
6.85
3.83
2.26
1.85
0.48
2.63
1.05
1.56
2.96
6.04
1.39
5.77
5.77
5.89
5.99
0.64
0.58
1.06
5.54
2.09
3.55
2.36
5.30
2.69
2.94
4.27
4.07
0.29
2.16
1.22
0.54
3.17
1.06
2.69
0.42
3.28
3.87
3.77
3.87
1.76
2.30
3.27
27.33
6.30
0.29
1.24
5.50
0.93
2.36
1.75
0.37
23.56
5.80
0.63
4.29
4.45
3.63
0.33
3.21
3.75
5.03
1.01
2.41
5.19
2.52
6.82
2.52
1.06
4.66
2.63
5.63
2.44
2.46
1.12
0.95
1.54
1.26
2.30
2.27
1.80
4.56
1.90
6.30
2.43
1.08
3.94
1.55
4.64
1.26
1.39
2.18
3.42
5.40
4.43
2.55
5.90
23.56
1.05
6.82
1.02
2.99
1.75
3.93
0.92
5.23
1.71
27.83
4.27
6.67
1.80
2.73
5.28
0.27
0.92
1.46
2.59
4.89
4.16
4.43
3.49
2.94
1.49
5.46
3.62
1.95
2.32
3.51
0.74
0.27
3.69
0.94
2.08
5.19
0.30
29.87
3.27
6.08
4.27
4.02
3.76
0.19
5.19
2.26
1.95
5.05
4.26
4.32
0.07
5.52
2.70
1.06
1.51
1.19
1.10
3.77
5.37
2.41
5.72
0.12
7.02
1.74
4.85
3.08
2.73
0.96
4.17
0.85
3.18


3.82
0.07
2.46
3.42
1.49
1.12
1.67
2.71
1.46
5.37
27.55
0.96
3.40
1.06
1.44
6.46
1.17
3.21
0.78
2.36
5.77
0.32
21.15
2.86
0.43
5.72
4.82
3.51
6.46
5.26
6.29
5.66
3.30
5.34
5.53
2.54
3.79
3.30
1.58
1.90
2.55
2.39
3.76
5.06
6.00
1.93
3.37
3.57
3.30
4.32
4.29
3.77
2.93
0.75
1.75
3.18
4.79
5.74
2.14
1.31
0.76
0.85
5.90
2.59
3.37
2.32
0.59
2.70
5.69
1.99
4.26
4.85
0.50
0.89
1.06
4.27
1.41
1.32
0.21
5.70
1.69
2.48
1.67
3.90
3.50
5.35
3.76
4.02
1.93
5.37
3.08
22.92
1.50
1.42
4.27
6.71
3.07
6.94
1.74
21.10
2.72
1.69
3.29
1.42
2.11
2.59
1.06
0.32
3.90
1.86
2.96
0.32
1.52
0.95
2.36
1.14
2.15
0.89
1.56
23.56
5.19
5.22
3.55
6.30
2.52
3.90
4.48
4.45
2.58
1.83
0.77
5.66
5.05
5.22
26.58
1.66
2.79
4.36
4.90
1.55
2.38
3.50
1.37
1.69
0.69
3.21
0.63
2.94
2.11
0.48
1.58
6.82
2.25
3.82
4.07
2.71
0.89
3.87
3.78
2.36
1.95
5.52
3.18
3.21
2.52
5.91
1.40
4.88
0.89
27.83
5.59
3.75
3.50
4.03
5.19
1.68
3.93
1.31
4.64
2.43
0.63
5.45
1.50
1.56
1.22
3.90
2.76
2.67
2.14
3.66
0.85
2.52
5.03
4.91
1.49
1.07
3.76
5.35
1.4

3.27
4.23
1.49
2.08
6.56
1.06
6.82
1.52
1.69
2.05
4.90
2.67
2.02
1.71
1.56
1.10
2.41
0.48
2.02
2.20
0.26
5.52
5.35
1.12
0.63
2.69
5.70
4.02
0.59
3.86
6.15
1.58
4.82
5.72
1.06
6.82
1.76
1.51
0.65
0.27
0.21
1.95
3.59
0.92
3.87
3.49
5.72
2.14
2.55
5.72
5.63
1.19
3.07
1.90
0.96
1.56
5.70
4.23
5.20
0.42
6.67
1.50
2.20
4.96
4.56
1.67
2.20
3.26
1.59
1.05
5.19
1.76
2.39
1.86
1.55
3.74
2.73
2.20
6.08
1.49
2.43
2.20
2.02
3.76
0.54
5.59
4.27
3.82
5.72
3.27
1.52
1.74
1.93
5.59
0.78
2.03
5.91
1.26
6.63
1.03
5.59
26.58
3.28
1.19
4.48
4.65
5.90
1.58
0.96
4.60
3.17
1.52
3.27
1.38
2.16
1.17
4.37
2.69
0.50
5.71
2.79
2.15
0.64
0.17
2.20
4.52
5.68
1.95
2.41
5.99
2.73
2.39
1.19
6.29
3.20
3.07
2.97
1.55
1.80
5.03
1.03
4.88
0.85
3.75
1.30
2.07
4.64
3.66
0.99
3.75
2.02
1.59
6.85
2.73
0.95
1.58
0.92
3.50
20.97
1.14
0.37
2.54
4.36
2.76
0.50
1.67
1.86
4.88
5.64
3.07
4.67
2.30
6.15
2.16
8.73
1.03
3.73
3.07
3.90
2.99
22.75
4.17
1.75
1.91
4.52
2.39
6.19
29.87
3.20
3.77
3.82
1.07
3.69
1.43
1.59
22.75
0.96
3.75
2.58


In [ ]:
transactions_ready['EPS_ratio']=l_EPS_ratios

In [ ]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment,ESG_ranking,PE_ratio,EPS_ratio
0,KSS,RETAIL,30,300,2016-07-11,2016-08-10,32.497158,31.311853,-0.036474,BAD,12.9,11.1,3.07
1,TYO,AUTO,300,20000,2017-01-05,2017-11-01,14.744580,13.657829,-0.073705,BAD,30.3,NaN,NaN
3,GM,AUTO,3,10000,2016-07-25,2016-07-28,27.413452,26.498533,-0.033375,BAD,30.6,3.1,7.81
5,MSFT,TECH,2,500,2015-06-26,2015-06-29,40.755909,39.954479,-0.019664,BAD,14.7,27.33,1.46
7,BBY,RETAIL,3,20000,2016-08-29,2016-09-01,34.658939,34.232590,-0.012301,BAD,12.0,9.71,3.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39991,FB,TECH,240,100,2016-07-27,2017-03-24,123.339996,140.339996,0.137830,GOOD,31.6,54.94,2.08
39993,KSS,RETAIL,80,1500,2018-07-25,2018-10-12,66.574463,65.971695,-0.009054,BAD,12.9,11.59,5.72
39994,AXP,BANK,180,1500,2016-05-06,2016-11-02,59.775597,61.258610,0.024810,GOOD,19.8,11.33,5
39996,NVDA,TECH,4,300,2016-03-11,2016-03-15,31.640415,31.561857,-0.002483,BAD,12.9,26.56,1.08


## PS ratio 

In [ ]:
ps_ratios_collection= pickle.load(open('ps_ratios_collection.p', "rb" ))
ps_ratios_collection['AAPL'].head(5)

,Date,Stock_Price,TTM_Sales_per_Share,Price_to_Sales_Ratio
0,NaT,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN
2,2021-01-01,127.14,,8.16
3,2020-09-01,115.61,$15.58,7.42
4,2020-06-01,90.88,$15.33,5.93
5,2020-03-01,63.18,$14.83,4.26
6,2019-12-01,72.78,$14.61,4.98
7,2019-09-01,55.35,$13.87,3.99
8,2019-06-01,48.72,$13.57,3.59
9,2019-03-01,46.58,$13.35,3.49


In [ ]:


l_PS_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=ps_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            PS_ratio_str=ps_ratios_collection[company][ps_ratios_collection[company]['Date']==date].Price_to_Sales_Ratio.values[0]
            PS_ratio=float(PS_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PS_ratio= Decimal('nan')
            
    except:
        PS_ratio=Decimal('nan')
    
    l_PS_ratios.append(PS_ratio)

    
transactions_ready['PS_ratio']=l_PS_ratios

In [ ]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment,ESG_ranking,PE_ratio,EPS_ratio,PS_ratio
0,KSS,RETAIL,30,300,2016-07-11,2016-08-10,32.497158,31.311853,-0.036474,BAD,12.9,11.1,3.07,0.34
1,TYO,AUTO,300,20000,2017-01-05,2017-11-01,14.744580,13.657829,-0.073705,BAD,30.3,NaN,NaN,NaN
3,GM,AUTO,3,10000,2016-07-25,2016-07-28,27.413452,26.498533,-0.033375,BAD,30.6,3.1,7.81,0.28
5,MSFT,TECH,2,500,2015-06-26,2015-06-29,40.755909,39.954479,-0.019664,BAD,14.7,27.33,1.46,3.53
7,BBY,RETAIL,3,20000,2016-08-29,2016-09-01,34.658939,34.232590,-0.012301,BAD,12.0,9.71,3.07,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39991,FB,TECH,240,100,2016-07-27,2017-03-24,123.339996,140.339996,0.137830,GOOD,31.6,54.94,2.08,14.89
39993,KSS,RETAIL,80,1500,2018-07-25,2018-10-12,66.574463,65.971695,-0.009054,BAD,12.9,11.59,5.72,0.54
39994,AXP,BANK,180,1500,2016-05-06,2016-11-02,59.775597,61.258610,0.024810,GOOD,19.8,11.33,5,1.63
39996,NVDA,TECH,4,300,2016-03-11,2016-03-15,31.640415,31.561857,-0.002483,BAD,12.9,26.56,1.08,3.23


## PB ratio 

In [ ]:
pb_ratios_collection= pickle.load(open('pb_ratios_collection.p', "rb" ))
pb_ratios_collection['AAPL'].head()

,Date,Stock_Price,Book_Value_per_Share,Price_to_Book_Ratio
0,NaT,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN
2,2021-01-01,127.14,,33.03
3,2020-09-01,115.61,$3.85,30.04
4,2020-06-01,90.88,$4.22,21.54


In [ ]:


l_PB_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=pb_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            PB_ratio_str=pb_ratios_collection[company][pb_ratios_collection[company]['Date']==date].Price_to_Book_Ratio.values[0]
            PB_ratio=float(PB_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PB_ratio= Decimal('nan')
            
    except:
        PB_ratio=Decimal('nan')
    
    l_PB_ratios.append(PB_ratio)

    
transactions_ready['PB_ratio']=l_PB_ratios

## NetProfitMargin ratio 

In [ ]:
NetProfitMargin_ratios_collection= pickle.load(open('NetProfitMargin_ratios_collection.p', "rb" ))
NetProfitMargin_ratios_collection['AAPL'].head()

,Date,TTM_Revenue,TTM_Net_Income,Net_Margin
0,NaT,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN
2,2020-09-01,$274.52B,$57.41B,20.91%
3,2020-06-01,$273.86B,$58.42B,21.33%
4,2020-03-01,$267.98B,$57.22B,21.35%


In [ ]:

l_NetProfitMargin_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=NetProfitMargin_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            NetProfitMargin_ratio_str=NetProfitMargin_ratios_collection[company][NetProfitMargin_ratios_collection[company]['Date']==date].Net_Margin.values[0]
            NetProfitMargin_ratio_str=trim.sub('',NetProfitMargin_ratio_str)
            print(NetProfitMargin_ratio_str)
            NetProfitMargin_ratio=float(NetProfitMargin_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            NetProfitMargin_ratio= Decimal('nan')
            
    except:
        NetProfitMargin_ratio=Decimal('nan')
    
    l_NetProfitMargin_ratios.append(NetProfitMargin_ratio)

    
transactions_ready['NetProfitMargin_ratio']=l_NetProfitMargin_ratios

In [ ]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment,ESG_ranking,PE_ratio,EPS_ratio,PS_ratio,PB_ratio,NetProfitMargin_ratio,current_ratio
0,KSS,RETAIL,30,300,2016-07-11,2016-08-10,32.497158,31.311853,-0.036474,BAD,12.9,11.1,3.07,0.34,1.18,3.02,NaN
1,TYO,AUTO,300,20000,2017-01-05,2017-11-01,14.744580,13.657829,-0.073705,BAD,30.3,NaN,NaN,NaN,NaN,NaN,NaN
3,GM,AUTO,3,10000,2016-07-25,2016-07-28,27.413452,26.498533,-0.033375,BAD,30.6,3.1,7.81,0.28,0.88,9.12,NaN
5,MSFT,TECH,2,500,2015-06-26,2015-06-29,40.755909,39.954479,-0.019664,BAD,14.7,27.33,1.46,3.53,4,13.03,NaN
7,BBY,RETAIL,3,20000,2016-08-29,2016-09-01,34.658939,34.232590,-0.012301,BAD,12.0,9.71,3.07,0.26,2.19,2.62,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39991,FB,TECH,240,100,2016-07-27,2017-03-24,123.339996,140.339996,0.137830,GOOD,31.6,54.94,2.08,14.89,6.49,29.12,NaN
39993,KSS,RETAIL,80,1500,2018-07-25,2018-10-12,66.574463,65.971695,-0.009054,BAD,12.9,11.59,5.72,0.54,2.02,4.67,NaN
39994,AXP,BANK,180,1500,2016-05-06,2016-11-02,59.775597,61.258610,0.024810,GOOD,19.8,11.33,5,1.63,2.6,14.29,NaN
39996,NVDA,TECH,4,300,2016-03-11,2016-03-15,31.640415,31.561857,-0.002483,BAD,12.9,26.56,1.08,3.23,3.46,12.26,NaN


## Current ratio 

In [ ]:
current_ratios_collection= pickle.load(open('current_ratios_collection.p', "rb" ))
current_ratios_collection['AAPL'].head()

,Date,Current_Assets,Current_Liabilities,Current_Ratio
0,NaT,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN
2,2020-09-01,$143.71B,$105.39B,1.36
3,2020-06-01,$140.07B,$95.32B,1.47
4,2020-03-01,$143.75B,$96.09B,1.50


In [ ]:


l_current_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=current_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            current_ratio_str=current_ratios_collection[company][current_ratios_collection[company]['Date']==date].Current_Ratio.values[0]
            current_ratio=float(current_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            current_ratio= Decimal('nan')
            
    except:
        current_ratio=Decimal('nan')
    
    l_current_ratios.append(current_ratio)

    
transactions_ready['current_ratio']=l_current_ratios

## ROA ratio 

In [ ]:
roa_ratios_collection= pickle.load(open('roa_ratios_collection.p', "rb" ))
roa_ratios_collection['AAPL'].head()

,Date,TTM_Net_Income,Total_Assets,Return_on_Assets
0,NaT,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN
2,2020-09-01,$57.41B,$323.89B,17.63%
3,2020-06-01,$58.42B,$317.34B,17.75%
4,2020-03-01,$57.22B,$320.40B,17.31%


In [ ]:


l_roa_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=roa_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            roa_ratio_str=roa_ratios_collection[company][roa_ratios_collection[company]['Date']==date].Return_on_Assets.values[0]
            roa_ratio_str=trim.sub('',roa_ratio_str)
            roa_ratio=float(roa_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            roa_ratio= Decimal('nan')
            
    except:
        roa_ratio=Decimal('nan')
    
    l_roa_ratios.append(roa_ratio)

    
transactions_ready['roa_ratio']=l_roa_ratios

## ROE ratio 

In [ ]:
roe_ratios_collection= pickle.load(open('roe_ratios_collection.p', "rb" ))
roe_ratios_collection['AAPL'].head()

,Date,TTM_Net_Income,Shareholder's_Equity,Return_on_Equity
0,NaT,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN
2,2020-09-01,$57.41B,$65.34B,75.15%
3,2020-06-01,$58.42B,$72.28B,70.66%
4,2020-03-01,$57.22B,$78.43B,64.49%


In [ ]:


l_roe_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=roe_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            roe_ratio_str=roe_ratios_collection[company][roe_ratios_collection[company]['Date']==date].Return_on_Equity.values[0]
            roe_ratio_str=trim.sub('',roe_ratio_str)
            roe_ratio=float(roe_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            roe_ratio= Decimal('nan')
            
    except:
        roe_ratio=Decimal('nan')
    
    l_roe_ratios.append(roe_ratio)

    
transactions_ready['roe_ratio']=l_roe_ratios

In [ ]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment,ESG_ranking,PE_ratio,EPS_ratio,PS_ratio,PB_ratio,NetProfitMargin_ratio,current_ratio,roa_ratio,roe_ratio
0,KSS,RETAIL,30,300,2016-07-11,2016-08-10,32.497158,31.311853,-0.036474,BAD,12.9,11.1,3.07,0.34,1.18,3.02,1.77,4.16,10.67
1,TYO,AUTO,300,20000,2017-01-05,2017-11-01,14.744580,13.657829,-0.073705,BAD,30.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GM,AUTO,3,10000,2016-07-25,2016-07-28,27.413452,26.498533,-0.033375,BAD,30.6,3.1,7.81,0.28,0.88,9.12,0.98,6.24,30.96
5,MSFT,TECH,2,500,2015-06-26,2015-06-29,40.755909,39.954479,-0.019664,BAD,14.7,27.33,1.46,3.53,4,13.03,2.47,7.01,13.85
7,BBY,RETAIL,3,20000,2016-08-29,2016-09-01,34.658939,34.232590,-0.012301,BAD,12.0,9.71,3.07,0.26,2.19,2.62,1.43,7.52,23.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39991,FB,TECH,240,100,2016-07-27,2017-03-24,123.339996,140.339996,0.137830,GOOD,31.6,54.94,2.08,14.89,6.49,29.12,12.22,12.67,14.09
39993,KSS,RETAIL,80,1500,2018-07-25,2018-10-12,66.574463,65.971695,-0.009054,BAD,12.9,11.59,5.72,0.54,2.02,4.67,1.82,7.16,17.89
39994,AXP,BANK,180,1500,2016-05-06,2016-11-02,59.775597,61.258610,0.024810,GOOD,19.8,11.33,5,1.63,2.6,14.29,1.92,3.13,23.37
39996,NVDA,TECH,4,300,2016-03-11,2016-03-15,31.640415,31.561857,-0.002483,BAD,12.9,26.56,1.08,3.23,3.46,12.26,2.58,8.52,13.9


In [ ]:
transactions_ready.to_csv('transactions_variables.csv')